In [1]:
import gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation as LDA
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
target_category = [
    38,  # アイスクリーム__ノベルティー
    110,  # スナック・キャンディー__ガム
    113,  # スナック・キャンディー__シリアル
    114,  # スナック・キャンディー__スナック
    134,  # チョコ・ビスクラ__チョコレート
    171,  # ビール系__RTD
    172,  # ビール系__ノンアルコール
    173,  # ビール系__ビール系
    376,  # 和菓子__米菓
    435,  # 大型PET__無糖茶（大型PET）
    467,  # 小型PET__コーヒー（小型PET）
    537,  # 水・炭酸水__大型PET（炭酸水）
    539,  # 水・炭酸水__小型PET（炭酸水）
    629,  # 缶飲料__コーヒー（缶）
    768,  # 麺類__カップ麺
]

In [5]:
# 2020-08-01以前で10分以上経過し購買が発生したセッションにtrainデータを絞る
tmp_sessions = meta[meta['date'] < '2020-08-01']['session_id'].unique()
tmp_log = cartlog[cartlog["session_id"].isin(tmp_sessions)]
print('2020-08-01以前: ', len(tmp_sessions))

# 購買が発生したセッション
payment_sessions = set(tmp_log[tmp_log['is_payment']==1]['session_id'].unique())
print('購買が発生: ', len(payment_sessions))
# 10分以上のセッション
over10min_sessions = set(tmp_log[tmp_log['spend_time']>=600]['session_id'].unique())
print('10分以上: ', len(over10min_sessions))
# 積集合
all_train_sessions = payment_sessions & over10min_sessions
print('積集合: ', len(all_train_sessions))

# 10分以上の全trainのログデータ
all_train_log = tmp_log[tmp_log["session_id"].isin(all_train_sessions)]
print('全trainのログ: ', len(all_train_log))

2020-08-01以前:  663708
購買が発生:  618462
10分以上:  404825
積集合:  391383
全trainのログ:  10826062


In [6]:
def agg_payment(cartlog) -> pd.DataFrame:
    """セッションごと・商品ごとの購買個数を集計する"""
    # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
    agg = cartlog.loc[cartlog["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    return pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner").drop(columns=['JAN'])

In [7]:
class RetailDataset:
    def __init__(self, thres_sec, meta):
        self.thres_sec = thres_sec
        self.meta = meta.copy()
        self.meta['time_elapsed_sec'] = self.meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        
        # all_train_logの中で、指定時間より前のログのみを抽出　-> public_train_log, train_sessions
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        self.public_train_log = merge_train[merge_train['spend_time'] <= merge_train['time_elapsed_sec']]
        self.train_sessions = self.public_train_log["session_id"].unique()
        
    def get_train_input_log(self) -> pd.DataFrame:
        return self.public_train_log

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        train_target = pd.DataFrame(
            index=self.train_sessions,
        )
        train_target.index.name = "session_id"

        # time_elapsed以降のデータから購買個数を集計する
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        after_elapsed_log = merge_train[merge_train['spend_time'] > merge_train['time_elapsed_sec']]
        
        train_item_num = agg_payment(after_elapsed_log)
        train_item_num = train_item_num[train_item_num['category_id'].isin(target_category)]
        train_target_pos = train_item_num.groupby(["session_id", "category_id"])["n_items"].sum().unstack().fillna(0).astype(int)
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        return train_target.join(train_target_pos).fillna(0).reset_index()

In [8]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [9]:
ELAPSED_MIN = [0, 3, 5, 10]
# ELAPSED_MIN = [5]

In [10]:
def save_train_log():
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train2_log_{}.f'.format(elap_min))
        train_y.to_feather('../inputs/train2_y_{}.f'.format(elap_min))

In [11]:
def load_train_log():
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train2_log_{}.f'.format(elap_min))
        y = pd.read_feather('../inputs/train2_y_{}.f'.format(elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [12]:
# save_train_log()

- 0, train_session (378594, 16)
- 3, train_session (389649, 16)
- 5, train_session (390621, 16)
- 10, train_session (391074, 16)

### 過去のログデータ

In [13]:
payed_item = agg_payment(all_train_log)

#### クーポン発行とターゲットカテゴリの購入数の関係

In [14]:
# def get_coupon_log_data(payed_item):
#     # train用のデータから購買した商品
#     payed_item_in_cat = payed_item[payed_item['category_id'].isin(target_category)]
#     cat_pivot = payed_item_in_cat.pivot_table(
#         index='session_id', columns='category_id', values='n_items').fillna(0).add_prefix('coupon_').reset_index()

#     # 発券されたクーポン
#     coupon = all_train_log[all_train_log['kind_1'] == 'クーポン']
#     coupon_count = coupon['value_1'].value_counts()
#     print(len(coupon_count))
#     session_coupon = coupon[coupon['value_1'].isin(list(coupon_count.index))][['session_id', 'value_1']]
#     session_coupon = session_coupon.rename(columns={'value_1':'coupon'})
#     session_coupon = session_coupon.groupby(['session_id', 'coupon']).size().reset_index()

#     session_cat_coupon = pd.merge(cat_pivot, session_coupon[['session_id', 'coupon']], on='session_id', how='left')
#     session_cat_coupon['coupon'] = session_cat_coupon['coupon'].fillna(0)
#     session_cat_coupon.drop(columns=['session_id'], inplace=True)
#     group_coupon = session_cat_coupon.groupby(['coupon']).mean()
#     group_coupon[group_coupon < 0] = 0
#     return group_coupon.reset_index()

In [15]:
coupon_name = {
    'バナナ', '乳', '鶏', 'ブレッド', 'えのき', '白菜', '豆腐', 'ちくわ', 'ゴールド', '茶', '玉子', 'ポテト', 'ハム', 'しめじ', '揚げ', 'レタス',
    'エッセン', '豚', 'チーズ', 'プレーン', 'おかめ', 'サラダ', 'とまと', 'ねぎ', '福岡', 'エビ', '無糖', 'みかん', '焼そば', 'ロール', 'えび', 'ヨーグルト', 
    'シーチキン', '玉葱', '柿の種', 'サブレ', '油', 'りんご', 'きゅうり', 'いなり', 'バター', 'チャップ', 'マヨネーズ', 'ピーマン', 'ネスレ', 'ぱん', 'パン', 'いか', 
    'フロスト', '人参', '蒲鉾', 'トイレ', '舞茸', 'チョコ', '鮭', 'コーラ', 'チップス', 'ごはん', 'うどん', 'ウインナー', 'かつお', '梨', 'のり', 'キウイ', 'バニラ', 
    'パイ', 'キャラメル', 'カルビー', 'キムチ', 'アルフォート', 'コーヒー', 'フルグラ', 'ゼリー', '脂肪', 'アソート', 'キットカット', 'ドデカミン', '伊右衛門', 'ミルク',
}

In [16]:
def coupon_category():
    coupon = all_train_log[all_train_log['kind_1'] == 'クーポン']
    coupon_count = coupon['name_1'].value_counts()
    print(len(coupon_count))
    df_coupon = pd.DataFrame(coupon_count).reset_index()

    for i, name in enumerate(coupon_name):
        df_coupon.loc[df_coupon['index'].str.contains(name), 'coup_cat'] = str(i)

    df_coupon = df_coupon[df_coupon['coup_cat'].notnull()]
    return df_coupon.rename(columns={'index':'coupon'})[['coupon', 'coup_cat']]

In [17]:
df_coupon_cat = coupon_category()

1927


In [18]:
def LDA_topic(df_input, topic, index, prefix):
    df_cp = df_input.set_index(index)
    lda = LDA(n_components=topic)
    lda_out = pd.DataFrame(lda.fit_transform(df_cp), index=df_cp.index).add_prefix(prefix)
    return lda_out.reset_index()

In [19]:
# group_coupon = get_coupon_log_data(payed_item)
# lda_coupon = LDA_topic(group_coupon, 6, 'coupon', 'LDA_coup_')

### ユーザ情報

In [20]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left")
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

In [21]:
def get_user_item(payed_item):
    # train用のデータから購買した商品
    user_payed_item = pd.merge(payed_item, meta[['session_id', 'user_id']], on='session_id', how='left')
    group_user_item = user_payed_item.groupby(['user_id', 'category_id'])[['n_items']].sum().reset_index()
    pivot_user_item = group_user_item.pivot_table(index='user_id', columns='category_id', values='n_items')
    # 全ユーザーの購入数の合計が5000以上のカテゴリに絞り込み
    sum_user_item = pivot_user_item.sum()
    user_item_index = sum_user_item[sum_user_item > 5000].index
    pivot_user_item = pivot_user_item[user_item_index].fillna(0).reset_index()
    # trainに存在しない人用に平均値で穴埋め
    user_item_mean = pivot_user_item.mean()
    # 全ユーザーとマージ
    all_user_item = pd.merge(user_master[['user_id']], pivot_user_item, on='user_id', how='left')
    # targetのカテゴリは除く
    for col in all_user_item.columns:
        if (col == 'user_id') or (col in target_category):
            continue
        new_col = 'user_pay_{}'.format(col)
        all_user_item[new_col] = all_user_item[col].fillna(user_item_mean[col]).astype('float32')
        all_user_item.loc[all_user_item[new_col]<0, new_col] = 0

    return all_user_item.drop(columns=list(user_item_mean.index))

In [22]:
all_user_item = get_user_item(payed_item)
print(all_user_item.shape)

(40350, 231)


In [23]:
# %%time
# user_lda = LDA_topic(all_user_item, 10, 'user_id', 'LDA_')
# user_lda.to_feather('../inputs/user_lda.f')

In [24]:
user_lda = pd.read_feather('../inputs/user_lda.f')
all_user_item = pd.merge(all_user_item, user_lda, on='user_id', how='left')

In [25]:
print(all_user_item.shape)

(40350, 241)


### メタ情報

In [26]:
def get_meta_features(meta):
    meta_features = meta.copy()
    meta_features['year'] = meta_features['date'].dt.year
    meta_features['month'] = meta_features['date'].dt.month
    meta_features['day'] = meta_features['date'].dt.day
    meta_features['dow'] = meta_features['date'].dt.dayofweek
    meta_features['doy'] = meta_features['date'].dt.dayofyear
    meta_features['week_time'] = meta_features['dow'] * 24 + meta_features['hour']
    le = preprocessing.LabelEncoder()
    meta_features['userid'] = le.fit_transform(meta_features['user_id'])
    
    # 曜日x時間の来店者数
    df_tz = meta_features.groupby(['week_time']).size().rename('timezone_count')
    df_tz =  pd.DataFrame(df_tz).reset_index()
    meta_features = pd.merge(meta_features, df_tz, on='week_time', how='left')
    
    # userごとに前の来店からどれくらいの日数が経ったか
    df = meta_features.groupby(['session_id', 'user_id', 'date']).first().reset_index().sort_values(['user_id', 'session_id'])
    df['date_diff'] = df['date'].diff(1)
    df['user_diff'] = df['user_id'].shift(1)
    df.loc[df['user_diff'] != df['user_id'], 'date_diff'] = pd.NaT
    df['date_diff'] = df['date_diff'].dt.days
    meta_features = meta_features.join(df[['date_diff']])

    # userごとの来店回数
    meta_features['date_rank'] = meta_features.groupby(['user_id'])['date'].rank(ascending=True)
    
    return meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'])

In [27]:
meta_features = get_meta_features(meta)

### ディスプレイアクション

In [28]:
disp_name_dic = {}
for i, disp in enumerate(display_action_id['display_name'].unique()):
    disp_name_dic[disp] = 'disp_cnt_{}'.format(i)
    
act_name_dic = {}
for i, action in enumerate(display_action_id['action_name'].unique()):
    act_name_dic[action] = 'act_cnt_{}'.format(i)

In [29]:
def get_display_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    disp_group_count = merge.groupby(['session_id', 'display_name']).size().reset_index().rename(columns={0:'disp_name_count'})
    disp_name_pivot = disp_group_count.pivot_table(index='session_id', columns='display_name', values='disp_name_count', aggfunc='sum')
    disp_name_pivot = disp_name_pivot.reset_index().fillna(0).rename(columns=disp_name_dic)
    
    disp_out = disp_name_pivot[['session_id']].copy()
    for val in disp_name_dic.values():
        disp_out[val] = 0
    
    for col in disp_name_pivot.columns:
        if col == 'session_id':
            continue
        disp_out[col] = disp_name_pivot[col]
    
    return disp_out

In [30]:
def get_action_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    act_group_count = merge.groupby(['session_id', 'action_name']).size().reset_index().rename(columns={0:'act_name_count'})
    act_name_pivot = act_group_count.pivot_table(index='session_id', columns='action_name', values='act_name_count', aggfunc='sum')
    act_name_pivot = act_name_pivot.reset_index().fillna(0).rename(columns=act_name_dic)
    
    act_out = act_name_pivot[['session_id']].copy()
    for val in act_name_dic.values():
        act_out[val] = 0
    
    for col in act_name_pivot.columns:
        if col == 'session_id':
            continue
        act_out[col] = act_name_pivot[col]
    
    return act_out

### セッション単位の特徴量

In [31]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'value_1']].rename(columns={'value_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, lda_coupon, on='coupon', how='left').drop(columns=['coupon'])
#     session_coupon = session_coupon.groupby(['session_id']).max().reset_index()
#     return session_coupon

In [32]:
def get_coupon_info(input_log):
    session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'name_1']].rename(columns = {'name_1':'coupon'})
    session_coupon = pd.merge(session_coupon, df_coupon_cat, on='coupon', how='left').drop(columns=['coupon'])
    
    for cat in df_coupon_cat['coup_cat'].unique():
        new_col = 'coup_cat_{}'.format(cat)
        session_coupon[new_col] = 0
        session_coupon.loc[session_coupon['coup_cat'] == cat, new_col] = 1
    
    session_coupon.drop(columns=['coup_cat'], inplace=True)
    return session_coupon.groupby('session_id').sum().reset_index()

In [33]:
def get_pre_payment_item(input_log):
    session_unique = input_log['session_id'].unique()
    agg = input_log.loc[input_log["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    agg = pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    agg = agg[agg['category_id'].isin(target_category)]
    agg = agg.groupby(["session_id", "category_id"])["n_items"].sum().reset_index()
    
    sesi = np.zeros(len(target_category))
    cate = [ct for ct in target_category]
    
    dummy = pd.DataFrame({'session_id':sesi, 'category_id':cate, 'n_items':sesi})
    agg = pd.concat([agg, dummy])
    
    agg = agg.pivot_table(index='session_id', columns='category_id', values='n_items').fillna(0)
    src_columns = ['x_{}'.format(c) for c in agg.columns]
    agg.columns = src_columns
    
    col = ['pre_target_{}'.format(c) for c in target_category]
    df_out = pd.DataFrame(index=session_unique, columns=col)
    df_out.index.name = "session_id"
    df_out = df_out.join(agg)
    for ct in target_category:
        src = 'x_{}'.format(ct)
        dst = 'pre_target_{}'.format(ct)
        df_out[dst] = df_out[src]
    
    return df_out.drop(columns=src_columns).fillna(0).reset_index()

In [34]:
def get_session_kind_group(input_log):
    kind_name ={
        'クーポン': 'coupon',
        '会計': 'kaikei',
        'キー': 'key',
        'カテゴリ': 'categry',
        'バーコードスキャン': 'barcode',
        'UUID': 'uuid',
        '使用ポイント': 'usedpoint',
        '確認': 'confirm',
        'ブランドスイッチ': 'bland',
        'レシピ': 'recipe',
        'スマホスキャン': 'smartphone',
        '磁気スキャン': 'magnetic',
        'レコメンド': 'recommend',
        '倍率ポイント': 'point',
    }
    group_count = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename('group_count_'+'item')
    for kind, name in kind_name.items():
        tmp = input_log[input_log['kind_1'] == kind].groupby(["session_id"]).size().rename('group_count_'+name)
        group_count = pd.concat([group_count, tmp], axis=1)
        
    return group_count.reset_index()

In [35]:
def get_session_info(input_log):
    # アクション数
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    # 経過時間の平均
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    
    session_features = pd.concat([
        n_actions,
        mean_spend_time,
    ], axis=1)
    return session_features.reset_index()

### セッション単位で集計

In [36]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    
    session_feat = [
        get_session_info(input_log),
        get_session_kind_group(input_log),
        get_display_name_feature(input_log),
        get_action_name_feature(input_log),
        get_pre_payment_item(input_log),
        get_coupon_info(input_log),
    ]
    
    for feat in session_feat:
        df_ses = pd.merge(df_ses, feat, on='session_id', how='outer')
        
    return df_ses

### 特徴量を集約する

In [37]:
def merge_features(input_log, session):
    feat_list = [
        get_session_features(input_log),
        user_features,
        meta_features,
    ]
    out = pd.DataFrame({"session_id": session})
    for feat in feat_list:
        out = pd.merge(out, feat, on="session_id", how="left")
        
    # userの情報
    out = pd.merge(out, all_user_item, on='user_id', how='left').drop(columns='user_id')

    assert len(session) == len(out)
    return out

In [38]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_features', train_features.shape)
    return train_features, y_train

In [39]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_features', test_features.shape)
    return test_features

In [40]:
n_fold = 4

In [41]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
#   'max_depth': 6,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

In [42]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [43]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [44]:
train_log_list, train_y_list = load_train_log()

In [45]:
gc.collect()

20

In [46]:
%%time
df_pred_all = pd.DataFrame()
df_score_all = pd.DataFrame(index=ELAPSED_MIN)
models_list_list = []

for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
        df_score_all.loc[elapsed_min, target] = score
        
    models_list_list.append(models_list)
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_features (378594, 412)
test_features (14277, 412)
---- id = 38 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[930]	valid_0's auc: 0.826131
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[873]	valid_0's auc: 0.82822
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[838]	valid_0's auc: 0.829771
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[938]	valid_0's auc: 0.828718
--- FINISHED \ whole score: 0.8282 ---
---- id = 110 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[277]	valid_0's auc: 0.799439
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[268]	valid_0's auc: 0.788167
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[192]	valid_0's auc: 0.801064
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[290]	valid_0's auc: 0.801453
--- FINISHED \ whole score: 0.7972 ---
---- id = 113 -----


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[657]	valid_0's auc: 0.834015
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[305]	valid_0's auc: 0.821141
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[592]	valid_0's auc: 0.830533
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[702]	valid_0's auc: 0.838608
--- FINISHED \ whole score: 0.8307 ---
---- id = 629 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[376]	valid_0's auc: 0.868954
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[385]	valid_0's auc: 0.868214
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[183]	valid_0's auc: 0.871112
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[291]	valid_0's auc: 0.882308
--- FINISHED \ whole score: 0.8693 ---
---- id = 768 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[515]	valid_0's auc: 0.730173
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[512]	valid_0's auc: 0.731827
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[586]	valid_0's auc: 0.726501
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[407]	valid_0's auc: 0.735579
--- FINISHED \ whole score: 0.7309 ---
56486
CPU times: user 1d 22h 34min 41s, sys: 25min 24s, total: 1d 23h 6s
Wall time: 2h 8min 53s


In [47]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.094643,0.015625,0.017001,0.355180,0.069377,0.063255,0.001227,0.057764,0.103481,0.014642,0.076926,0.001692,0.057854,0.039597,0.018600
663761,0.205657,0.090208,0.432833,0.508659,0.280001,0.195516,0.558165,0.026732,0.237042,0.004007,0.009743,0.006482,0.321022,0.001656,0.496610
663763,0.161348,0.063763,0.091986,0.828784,0.509822,0.009552,0.003068,0.004342,0.237736,0.036590,0.053235,0.002092,0.008883,0.013125,0.053889
663775,0.034102,0.014397,0.014601,0.158829,0.297044,0.128764,0.013532,0.623097,0.048680,0.197361,0.023029,0.005531,0.038842,0.008180,0.199592
663778,0.278714,0.028037,0.124855,0.212330,0.155835,0.079865,0.008307,0.063698,0.041704,0.102811,0.004546,0.022661,0.058833,0.001202,0.082798


In [48]:
df_score_all

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0.828192,0.797225,0.802864,0.760580,0.747055,0.869483,0.870970,0.865886,0.757123,0.869634,0.871702,0.906037,0.859858,0.919209,0.748746
3,0.824458,0.787084,0.798146,0.755068,0.741429,0.862804,0.865368,0.858533,0.753009,0.855413,0.863289,0.899378,0.848574,0.910531,0.736269
5,0.824020,0.778722,0.792398,0.750661,0.737144,0.857250,0.857622,0.853783,0.750078,0.850109,0.855909,0.892641,0.843102,0.896734,0.731549
10,0.820812,0.770578,0.789388,0.755577,0.746653,0.842933,0.833284,0.847959,0.759134,0.838537,0.839050,0.879823,0.830698,0.869258,0.730873


In [49]:
cv = df_score_all.mean(axis=1)
print(cv)
print('- cv =', cv.mean())

0     0.831638
3     0.823957
5     0.818115
10    0.810304
dtype: float64
- cv = 0.821003295043145


In [50]:
assert len(df_pred_all) == len(test)

In [51]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [52]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)

#### baseline_10: クーポンカテゴリの表示回数
- feat = 412
- Wall time: 2h 8min 53s
- cv = 0.821003 (0.831/0.823/0.818/0.8103)
- LB = 0.7657

#### baseline_9-1: 'feature_fraction': 0.6,'bagging_fraction': 0.6,'bagging_freq': 2,
- feat = 333
- Wall time: 2h 2min 14s
- cv = 0.81961 (0.829/0.822/0.816/0.809)
- LB = 0.7658

#### baseline_9: 来店間隔、回数、曜日x時間の来店者数
- feat = 333
- Wall time: 2h 9min 51s
- cv = 0.82118 (0.831/0.824/0.818/0.810)
- LB = 0.7665

#### baseline_8: クーポン発行とターゲットの購買実績
- feat = 344
- Wall time: 2h 5min 13s
- cv = 0.820398 (0.832/0.822/0.817/0.809)
- LB = 0.7484

#### baseline_7: userごとのカテゴリの過去の購買実績
- feat = 329
- Wall time: 1h 59min 16s
- cv = 0.82148 (0.832/0.824/0.818/0.810)
- LB = 0.7671

#### baseline_6: sesssion内のターゲットの購買
- feat = 89
- Wall time: 37min 47s
- cv = 0.66015
- LB = 0.6247

#### baseline_5
- feat = 74
- Wall time: 39min 18s
- cv = 0.66859
- LB = 0.6230